In [1]:
import hopsworks

proj = hopsworks.login()
fs = proj.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://d870ee90-691a-11ee-8150-c9a8e99989f0.cloud.hopsworks.ai:443/p/2167
Connected. Call `.close()` to terminate connection gracefully.


In [2]:
import pandas as pd

df = pd.read_csv("../../data/ecommerce/order_item.csv")
df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [3]:
order_item_fg = fs.create_feature_group(
                        name="order_item_fg",
                        primary_key=["order_id", "order_item_id"],
                        version=1,
                        description="Order items",
                        online_enabled=True
)

order_item_fg.insert(df)

Feature Group created successfully, explore it at 
https://d870ee90-691a-11ee-8150-c9a8e99989f0.cloud.hopsworks.ai:443/p/2167/fs/2115/fg/1039


Uploading Dataframe: 0.00% |          | Rows 0/112650 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: order_item_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://d870ee90-691a-11ee-8150-c9a8e99989f0.cloud.hopsworks.ai/p/2167/jobs/named/order_item_fg_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f2f3315f850>, None)

In [7]:
product_fg = fs.get_feature_group("product_fg", version=1)

query = order_item_fg.select_all().join(product_fg.select_all(), on="product_id")

feature_view = fs.create_feature_view(
    name="product_orders",
    description="Training data for a products and orders model",
    labels=["price"],
    query=query
)

Feature view created successfully, explore it at 
https://d870ee90-691a-11ee-8150-c9a8e99989f0.cloud.hopsworks.ai:443/p/2167/fs/2115/fv/product_orders/version/1


In [8]:
df = query.read()
df

Finished: Reading data from Hopsworks, using Hive (46.21s) 


In [9]:
df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_len,product_description_len,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,c70ddc702499e20988c929a93a3f1c57,3,afe60f026d04f54945ecf4c085eff7f5,8e6d7754bc7e0f22c96d255ebda59eba,2018-01-11 15:56:24,199.89,17.15,informatica_acessorios,56.0,235.0,1.0,633.0,26.0,13.0,12.0
1,01da5549f136167573c8a302f008557a,1,2381a3052dd37f97e616bf160467acae,6b536a23086fba0d58c08d68814397f6,2018-04-16 03:55:21,150.00,20.02,relogios_presentes,39.0,615.0,1.0,530.0,18.0,9.0,13.0
2,262ba6c85fd3c9c589bac6fa7d113cca,1,a9008450835cd38eabdf07eb6dd04bdd,620c87c171fb2a6dd6e8bb4dec959fc6,2018-03-08 09:28:51,119.90,15.59,perfumaria,37.0,813.0,2.0,526.0,22.0,15.0,17.0
3,2b40e7763ce2a2fbeb3e63dd4a498702,1,15859e3a8aaa3c192c10a91d8722abd5,e5a38146df062edaf55c38afa99e42dc,2018-01-31 02:57:58,299.90,26.65,beleza_saude,51.0,2795.0,5.0,5650.0,36.0,26.0,36.0
4,ddba9859eaa6e81ea4f2ed0263d29291,1,a272b490eba6b022384f14ecb9afc7a9,82cb9f452fe8334df74b22d37824b431,2018-06-28 13:18:39,49.90,19.54,eletroportateis,54.0,987.0,1.0,525.0,32.0,18.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111041,81b2c7683a251dc0ea48059bef94597c,1,cce3ce6484896041ae3dafdd4308a39c,7ddcbb64b5bc1ef36ca8c151f6ec77df,2017-07-28 11:43:23,1004.99,33.57,consoles_games,46.0,653.0,2.0,4350.0,18.0,33.0,39.0
111042,f7b7cd620fccb9f2041e9f3b569d49e0,1,9927048351cd36357bd4227c238f478b,0ea22c1cfbdc755f86b9b54b39c16043,2017-10-20 01:28:21,54.90,15.13,musica,48.0,432.0,2.0,150.0,16.0,9.0,11.0
111043,156824260a71270937eda941cd291a58,1,fe6a9515d655fa7936b8a7c841039f34,dc317f341ab0e22f39acbd9dbf9b4a1f,2017-10-19 19:28:14,249.90,74.70,cool_stuff,29.0,266.0,3.0,5650.0,73.0,73.0,20.0
111044,6ee93b768b2ece278852a25fecf3eb03,2,e27c4dd075a521097a085bced0631ce7,701938c450705b8ae65fc923b70f35c7,2017-06-15 18:22:37,103.97,25.04,moveis_sala,31.0,618.0,2.0,9750.0,27.0,15.0,92.0


In [ ]:
# Here, we use the built-in storage connector to Hopsworks that will store the training data in Hopsworks.
# Hopsworks includes "hopsfs", a scale out file system with a HDFS API and a object storage backend.
# You can create your own storage connector to save your training data in a different storage backend.

hopsfs = fs.get_storage_connector("fsbook_Training_Datasets")

# This will take a few mins to run, as it will spin up a Spark job in Hopsworks to join the features
# together and write out the training data as files in .tfrecord file format (TensorFlow Records)
# on the file system in Hopsworks.
feature_view.create_training_data(
    version=1,
    connector=hopsfs,
    data_format="tfrecord"
)